In [1]:
import pandas as pd
import requests
import random
import json
import time
import glob
import os


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


In [9]:
tool2tag = {
    'Amazon SageMaker': {'amazon-sagemaker', 'amazon-sagemaker-experiments'},
    'Azure Machine Learning': {'azure-machine-learning-service', 'azure-machine-learning-studio', 'azure-machine-learning-workbench'},
    'ClearML': {'clearml'},
    'Comet': {'comet-ml'},
    'DVC': {'dvc'},
    'Kedro': {'kedro'},
    'MLFlow': {'mlflow'},
    'MLRun': {'mlrun'},
    'Neptune': {'neptune'},
    'Pachyderm': {'pachyderm'},
    'Sacred': {'python-sacred'},
    'Vertex AI': {'google-cloud-vertex-ai', 'vertex-ai-pipeline'},
    'Weights & Biases': {'wandb'}
}

ignore_tools = {
    'Amazon SageMaker',
    'Azure Machine Learning',
    'Vertex AI'
}


In [4]:
path_dataset = '../Dataset'

path_so = os.path.join(path_dataset, 'Stack Overflow')
path_to = os.path.join(path_dataset, 'Tool-specific Others')

path_so_raw = os.path.join(path_so, 'Raw')
path_to_raw = os.path.join(path_to, 'Raw')
path_so_filtered = os.path.join(path_so, 'Filtered')
path_to_filtered = os.path.join(path_to, 'Filtered')
path_so_sampled = os.path.join(path_so, 'Sampled')
path_to_sampled = os.path.join(path_to, 'Sampled')

if not os.path.exists(path_dataset):
    os.makedirs(path_dataset)

if not os.path.isdir(path_so):
    os.mkdir(path_so)

if not os.path.isdir(path_to):
    os.mkdir(path_to)

if not os.path.isdir(path_so_raw):
    os.mkdir(path_so_raw)

if not os.path.isdir(path_to_raw):
    os.mkdir(path_to_raw)

if not os.path.isdir(path_so_filtered):
    os.mkdir(path_so_filtered)

if not os.path.isdir(path_to_filtered):
    os.mkdir(path_to_filtered)

if not os.path.isdir(path_so_sampled):
    os.mkdir(path_so_sampled)

if not os.path.isdir(path_to_sampled):
    os.mkdir(path_to_sampled)


In [8]:
page = -1
base_url = 'https://my.guild.ai/'
page_suffix = 'c/troubleshooting/6.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_time'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'Guild AI.json'), 'w') as outfile:
    outfile.write(json_post_list)


In [7]:
page = -1
base_url = 'https://community.wandb.ai/'
page_suffix = 'c/w-b-support/36.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_time'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'Weights & Biases.json'), 'w') as outfile:
    outfile.write(json_post_list)


In [5]:
page = -1
base_url = 'https://community.sigopt.com/'
page_suffix = 'c/general-discussion/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_time'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'SigOpt.json'), 'w') as outfile:
    outfile.write(json_post_list)


In [5]:
page = -1
base_url = 'https://discuss.dvc.org/'
page_suffix = 'c/questions/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_time'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'DVC.json'), 'w') as outfile:
    outfile.write(json_post_list)


In [28]:
df_to = pd.DataFrame()
# exclude tool-specific posts with negative upvote count
for file_name in glob.glob(os.path.join(path_to_raw, '*.json')):
    repos = pd.read_json(file_name)
    if 'Question_upvote_count' in repos.columns:
        repos = repos[repos['Question_upvote_count'] > -1]
    repos['Tool'] = os.path.split(file_name)[1].split('.')[0]
    df_to = pd.concat([df_to, repos], ignore_index=True)
df_to.to_json(os.path.join(path_to_filtered,
              'non_negative_scored.json'), orient='records', indent=4)

In [29]:
df_to_answered = df_to[df_to['Question_answer_count'] > 0]
for tool in df_to_answered['Tool'].unique().tolist():
    number_accepted_answer = df_to_answered[df_to_answered['Tool']
                                            == tool]['Question_has_accepted_answer'].sum()
    if number_accepted_answer > 0:
        df_to_answered = df_to_answered.drop(df_to_answered[(df_to_answered['Tool'] == tool) & (
            df_to_answered['Question_has_accepted_answer'] == False)].index)
df_to_answered.to_json(os.path.join(
    path_to_filtered, 'completed_non_negative_scored.json'), orient='records', indent=4)

In [94]:
# keep only posts with at least one answer
df_to_summary = df_to.groupby('Tool').count()['Question_title'].reset_index()
df_to_answered_summary = df_to_answered.groupby(
    'Tool').count()['Question_title'].reset_index()

df_to_summary.columns = ['Tool', '#Question']
df_to_answered_summary.columns = ['Tool', '#Answered']

df_summary = df_to_summary.merge(df_to_answered_summary, on='Tool')

df_summary.to_csv(os.path.join(path_to, 'summary.csv'), index=False)
df_summary


,Tool,#Question,#Answered
0,Amazon SageMaker,528,167
1,Azure Machine Learning,1435,343
2,DVC,315,300
3,Domino,13,4
4,Guild AI,115,108
5,MLFlow,280,143
6,Polyaxon,43,34
7,SigOpt,15,7
8,Vertex AI,297,32
9,Weights & Biases,583,92


In [10]:
# After having the population for each tool and discussion channel, we then find out the minimum number of necessary samples with the [calculator](https://www.calculator.net/sample-size-calculator.html).
df_summary = pd.read_csv(os.path.join(path_to, 'summary.csv'))
df_summary

,Tool,#Question,#Answered,#Sample Question,#Sample Answered
0,Amazon SageMaker,528,167,223,117
1,Azure Machine Learning,1435,343,304,182
2,DVC,315,300,174,169
3,Domino,13,4,13,4
4,Guild AI,115,108,89,85
5,MLFlow,280,143,163,105
6,Polyaxon,43,34,39,32
7,SigOpt,15,7,15,7
8,Vertex AI,297,32,168,30
9,Weights & Biases,583,92,232,75


In [17]:
df_to = pd.read_json(os.path.join(
    path_to_filtered, 'non_negative_scored.json'))
df_to_answered = pd.read_json(os.path.join(
    path_to_filtered, 'completed_non_negative_scored.json'))

In [18]:
# sample tool-specific posts accordingly
df_question_samples = pd.DataFrame()
df_answer_samples = pd.DataFrame()

for index, row in df_summary.iterrows():
    df_question_sample = df_to[df_to['Tool'] == row['Tool']].sample(
        n=row['#Sample Question'], random_state=0)
    df_answer_sample = df_to_answered[df_to_answered['Tool'] == row['Tool']].sample(
        n=row['#Sample Answered'], random_state=0)
    df_question_samples = pd.concat(
        [df_question_samples, df_question_sample], ignore_index=True)
    df_answer_samples = pd.concat(
        [df_answer_samples, df_answer_sample], ignore_index=True)

df_question_samples.to_json(os.path.join(
    path_to_sampled, 'questions.json'), indent=4, orient='records')
df_answer_samples.to_json(os.path.join(
    path_to_sampled, 'answers.json'), indent=4, orient='records')

In [27]:
df_question_samples = pd.read_json(
    os.path.join(path_to_sampled, 'questions.json'))
df_answer_samples = pd.read_json(os.path.join(path_to_sampled, 'answers.json'))
df_question_samples.loc[lambda x: ~x['Tool'].isin(ignore_tools)].to_json(
    os.path.join(path_to_sampled, 'lean_questions.json'), indent=4, orient='records')
df_answer_samples.loc[lambda x: ~x['Tool'].isin(ignore_tools)].to_json(
    os.path.join(path_to_sampled, 'lean_answers.json'), indent=4, orient='records')


In [11]:
df = pd.read_json(os.path.join(
    path_so_raw, 'bq-results-20230116-040400-1673841891210.json'), lines=True)
df.head()

,Question_id,Question_title,Question_body,Question_answer_count,Question_comment_count,Question_creation_time,Question_last_edit_time,Question_score,Question_tags,Question_view_count,...,Owner_reputation,Owner_up_votes,Owner_down_votes,Owner_views,Answer_body,Answer_comment_count,Answer_creation_time,Answer_last_edit_time,Answer_score,Question_favorite_count
0,813663,Moving from SVN to ...?,<p>I'm currently working in a team where we're...,9,2,2009-05-01 23:10:59.753000+00:00,2009-09-03 16:11:28.613000+00:00,4,svn|git|version-control|mercurial|dvcs,779,...,8487,152,6,796,<p>If your team can't understand how to use su...,0.0,2009-05-02 01:43:50.463000+00:00,2009-05-02 18:09:13.013000+00:00,20.0,NaN
1,960701,How can I always know about all tags in Mercur...,"<p>I don't use Mercurial, but i'd like to star...",2,1,2009-06-06 22:27:13.100000+00:00,2012-01-31 22:15:15.020000+00:00,16,mercurial|tags|dvcs,3863,...,45209,1615,111,5992,<p>Versioning the <code>.hgtags</code> file al...,5.0,2009-06-07 15:11:01.863000+00:00,NaT,20.0,7.0
2,10284208,"Getting mercurial ""hg commit"" to work with Not...",<p>I just replaced notepad with notepad++ usin...,1,1,2012-04-23 16:12:40.040000+00:00,NaT,14,mercurial|notepad++|dvcs,1802,...,20932,3469,7,2823,<p>Try placing in your <code>%USERPROFILE%\.hg...,3.0,2012-04-23 18:45:53.653000+00:00,NaT,21.0,1.0
3,51731086,Gremlin Coalesce To Add Multiple Vertices and ...,<p>Right now I am able to generate a query to ...,1,8,2018-08-07 16:02:32.170000+00:00,2021-07-09 09:48:56.313000+00:00,6,gremlin|tinkerpop3|amazon-neptune,5104,...,95,2,0,22,<p>You have a <code>fold()</code> which is a <...,5.0,2018-08-07 16:21:29.287000+00:00,2018-08-08 13:55:47.747000+00:00,11.0,2.0
4,10236112,What is a practical workflow for keeping local...,<p>What I want to do is already described in <...,1,2,2012-04-19 20:21:09.657000+00:00,2019-03-07 14:34:12.250000+00:00,10,git|version-control|dvcs,270,...,5795,1180,109,967,<p>You could try doing the following before yo...,3.0,2012-04-19 20:26:55.353000+00:00,2020-06-20 09:12:55.060000+00:00,10.0,2.0


In [12]:
# create tag collection
tags = []
for key, value in tool2tag.items():
    tags.extend(value)


In [13]:
# split tags
df['Question_valid_tags'] = [[] for _ in range(len(df))]
for index, row in df.iterrows():
    row_tags = set(str(row['Question_tags']).strip().split('|'))
    df.at[index, 'Question_valid_tags'] = list(row_tags.intersection(tags))

In [14]:
# count post number with different tags
arity = 0
while True:
    post_number = df[df['Question_valid_tags'].map(len) > arity].shape[0]
    if post_number < 1:
        break
    arity = arity + 1
    print(f'Posts with at least {arity} tags has {post_number} in total.')


Posts with at least 1 tags has 5136 in total.
Posts with at least 2 tags has 219 in total.
Posts with at least 3 tags has 18 in total.


In [15]:
# exclude Stack Overflow posts with unrelated tags
df_valid = df[df['Question_valid_tags'].map(len) > 0]
df_valid['Question_link'] = df_valid['Question_id'].apply(
    lambda x: f'https://stackoverflow.com/questions/{x}')

C:\Users\zhimi\AppData\Local\Temp\ipykernel_11552\105066014.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['Question_link'] = df_valid['Question_id'].apply(


In [16]:
# exclude Stack Overflow posts with negative upvote count
df_qualified = df_valid[df_valid['Question_score'] > -1]


In [17]:
# create a map from tag to tool
tag2tool = {}
for key, value in tool2tag.items():
    for elem in value:
        tag2tool.setdefault(elem, key)


'Amazon SageMaker'

'Amazon SageMaker'

'Azure Machine Learning'

'Azure Machine Learning'

'Azure Machine Learning'

...

'Pachyderm'

'Sacred'

'Vertex AI'

'Vertex AI'

'Weights & Biases'

In [18]:
# extract Stack Overflow post collection with multiple tags based on the tool map
for index, row in df_qualified.iterrows():
    tags = set()
    for tag in row['Question_valid_tags']:
        tags.add(tag2tool[tag])
    df_qualified.at[index, 'Question_valid_tags'] = sorted(list(tags))

In [19]:
# check how the posts with more than one tags look like
df_multiply_tagged = df_qualified[df_qualified['Question_valid_tags'].map(
    len) > 1]
df_multiply_tagged['Question_valid_tags'].value_counts()

[Amazon SageMaker, MLFlow]                 16
[Azure Machine Learning, MLFlow]           11
[Kedro, MLFlow]                             4
[Azure Machine Learning, Kedro, MLFlow]     2
[MLFlow, Sacred]                            1
[DVC, Pachyderm]                            1
[Kedro, Neptune]                            1
[DVC, MLFlow]                               1
Name: Question_valid_tags, dtype: int64

In [20]:
# create Stack Overflow post collection with exclusive tags
multiply_tagged_posts_split = []
df_qualified.assign(Question_exclusive_tag='')

for index, row in df_qualified.iterrows():
    tags = row['Question_valid_tags']
    df_qualified.at[index, 'Question_exclusive_tag'] = tags[0]
    if len(tags) > 1:
        for tag in tags[1:]:
            series = row.copy()
            series['Question_exclusive_tag'] = tag
            multiply_tagged_posts_split.append(series)

df_multiply_tagged_posts_split = pd.DataFrame(multiply_tagged_posts_split)
df_qualified_exclusive_tagged = pd.concat(
    [df_qualified, df_multiply_tagged_posts_split], ignore_index=True)
del df_qualified_exclusive_tagged['Question_valid_tags']
df_qualified_exclusive_tagged.to_json(os.path.join(
    path_so_filtered, 'non_negative_scored_exclusively_tagged.json'), indent=4, orient='records')

,Question_id,Question_title,Question_body,Question_answer_count,Question_comment_count,Question_creation_time,Question_last_edit_time,Question_score,Question_tags,Question_view_count,...,Owner_views,Answer_body,Answer_comment_count,Answer_creation_time,Answer_last_edit_time,Answer_score,Question_favorite_count,Question_valid_tags,Question_link,Question_exclusive_tag
7,65112585,Pip installation stuck in infinite loop if unr...,<p>Pip installation is stuck in an infinite lo...,1,6,2020-12-02 16:54:05.387000+00:00,2020-12-23 16:06:48.113000+00:00,13,python|pip|azure-machine-learning-service,2146,...,3,<p>Workarounds:</p>\n<p>Local environment:\nDo...,2.0,2020-12-02 17:02:08.563000+00:00,NaT,12.0,NaN,[Azure Machine Learning],https://stackoverflow.com/questions/65112585,
8,69024005,How to use SageMaker Estimator for model train...,<p>The documentations of how to use SageMaker ...,1,0,2021-09-02 04:01:47.170000+00:00,NaT,27,amazon-web-services|amazon-sagemaker,6655,...,968,<h1>Answer</h1>\n<p>There is no one such resou...,4.0,2021-09-02 04:01:47.170000+00:00,2022-07-04 05:43:30.063000+00:00,65.0,14.0,[Amazon SageMaker],https://stackoverflow.com/questions/69024005,
16,53609409,"Automatically ""stop"" Sagemaker notebook instan...",<p>I have a Sagemaker Jupyter notebook instanc...,4,0,2018-12-04 09:18:11.383000+00:00,NaT,19,amazon-web-services|aws-lambda|amazon-cloudwat...,12683,...,167,"<p>You can use <a href=""https://aws.amazon.com...",1.0,2019-10-15 09:46:13.200000+00:00,NaT,26.0,2.0,[Amazon SageMaker],https://stackoverflow.com/questions/53609409,
18,62422682,sagemaker notebook instance Elastic Inference ...,"<p>I am trying to replicate <a href=""https://g...",1,0,2020-06-17 06:25:29.670000+00:00,NaT,2,tensorflow|amazon-sagemaker,350,...,62,<p>Solved it. The error I was getting is due t...,2.0,2020-06-23 01:37:54.340000+00:00,NaT,-1.0,NaN,[Amazon SageMaker],https://stackoverflow.com/questions/62422682,
31,60525454,Intel optimized Python on Machine Learning Ser...,<p>Is it possible to run a Python script or Es...,1,0,2020-03-04 11:54:09.310000+00:00,2020-03-04 20:46:17.670000+00:00,0,azure-machine-learning-service,81,...,3,"<p>The Azure ML base images use <a href=""https...",5.0,2020-03-04 18:14:24.733000+00:00,2020-03-05 19:46:14.647000+00:00,-1.0,NaN,[Azure Machine Learning],https://stackoverflow.com/questions/60525454,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6793,34990561,Azure Machine Learning Request Response latency,<p>I have made an Azure Machine Learning Exper...,1,0,2016-01-25 10:40:39.993000+00:00,2016-01-27 16:15:36.527000+00:00,8,python|azure|azure-machine-learning-studio,1128,...,34,"<p>First, I am assuming you are doing your tim...",11.0,2016-01-26 18:20:06.127000+00:00,2016-01-27 16:10:48.927000+00:00,8.0,5.0,[Azure Machine Learning],https://stackoverflow.com/questions/34990561,
6801,49977679,upload data to S3 with sagemaker,<p>I have a problem with SageMaker when I try ...,1,0,2018-04-23 09:31:28.687000+00:00,2018-12-06 22:53:27.193000+00:00,5,amazon-s3|amazon-sagemaker,9575,...,108,"<p>It is exactly as the error say, the variabl...",1.0,2018-04-23 16:25:22.233000+00:00,NaT,8.0,1.0,[Amazon SageMaker],https://stackoverflow.com/questions/49977679,
6803,55580232,Update SageMaker Jupyterlab environment,<p>How can I update my SageMaker notebook's ju...,1,0,2019-04-08 19:10:35.030000+00:00,NaT,4,amazon-sagemaker|jupyter-lab,1720,...,958,<p>Hi and thank you for using SageMaker!</p>\n...,1.0,2019-04-11 18:28:36.017000+00:00,NaT,8.0,NaN,[Amazon SageMaker],https://stackoverflow.com/questions/55580232,
6806,58956459,How to run authentication on a mlFlow server?,<p>As I am logging my entire models and params...,4,1,2019-11-20 14:16:40.087000+00:00,NaT,11,nginx|basic-authentication|mlflow,13870,...,118,<p>the problem here is that both <code>mlflow<...,3.0,2019-12-13 16:37:32.617000+00:00,NaT,8.0,3.0,[MLFlow],https://stackoverflow.com/questions/58956459,


C:\Users\zhimi\AppData\Local\Temp\ipykernel_11552\3348297299.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qualified.at[index, 'Question_exclusive_tag'] = tags[0]


In [21]:
# keep Stack Overflow posts with accepted answers
df_qualified_exclusive_tagged_completed = df_qualified_exclusive_tagged.dropna(
    subset=['Answer_body'])
df_qualified_exclusive_tagged_completed.to_json(os.path.join(
    path_so_filtered, 'completed_non_negative_scored_exclusively_tagged.json'), indent=4, orient='records')


In [22]:
df_so_summary = df_qualified_exclusive_tagged.groupby(
    'Question_exclusive_tag').count()['Question_id'].reset_index()
df_so_answered_summary = df_qualified_exclusive_tagged_completed.groupby(
    'Question_exclusive_tag').count()['Question_id'].reset_index()

df_so_summary.columns = ['Tool', '#Question']
df_so_answered_summary.columns = ['Tool', '#Answered']

df_summary = pd.merge(df_so_summary, df_so_answered_summary, on='Tool')
df_summary.to_csv(os.path.join(path_so, 'summary.csv'), index=False)
df_summary


,Tool,#Question,#Answered
0,Amazon SageMaker,2233,737
1,Azure Machine Learning,1530,586
2,ClearML,40,20
3,Comet,10,4
4,DVC,91,49
5,Kedro,149,60
6,MLFlow,551,129
7,Neptune,8,3
8,Pachyderm,7,2
9,Sacred,10,7


In [23]:
# After having the population for each tool and discussion channel, we then find out the minimum number of necessary samples with the [calculator](https://www.calculator.net/sample-size-calculator.html).
df_summary = pd.read_csv(os.path.join(path_so, 'summary.csv'))
df_summary

,Tool,#Question,#Answered,#Sample Question,#Sample Answered
0,Amazon SageMaker,2233,737,328,253
1,Azure Machine Learning,1530,586,308,233
2,ClearML,40,20,37,20
3,Comet,10,4,10,4
4,DVC,91,49,74,44
5,Kedro,149,60,108,53
6,MLFlow,551,129,227,97
7,Neptune,8,3,8,3
8,Pachyderm,7,2,7,2
9,Sacred,10,7,10,7


In [21]:
df_qualified_exclusive_tagged = pd.read_json(os.path.join(
    path_so_filtered, 'non_negative_scored_exclusively_tagged.json'))
df_qualified_exclusive_tagged_completed = pd.read_json(os.path.join(
    path_so_filtered, 'completed_non_negative_scored_exclusively_tagged.json'))

In [15]:
# sample Stack Overflow posts accordingly
df_question_samples = pd.DataFrame()
df_answer_samples = pd.DataFrame()

for index, row in df_summary.iterrows():
    df_question_sample = df_qualified_exclusive_tagged[df_qualified_exclusive_tagged['Question_exclusive_tag'] == row['Tool']].sample(
        n=row['#Sample Question'], random_state=0)
    df_answer_sample = df_qualified_exclusive_tagged_completed[df_qualified_exclusive_tagged_completed['Question_exclusive_tag'] == row['Tool']].sample(
        n=row['#Sample Answered'], random_state=0)
    df_question_samples = pd.concat(
        [df_question_samples, df_question_sample], ignore_index=True)
    df_answer_samples = pd.concat(
        [df_answer_samples, df_answer_sample], ignore_index=True)

df_question_samples.to_json(os.path.join(
    path_so_sampled, 'questions.json'), indent=4, orient='records')
df_answer_samples.to_json(os.path.join(
    path_so_sampled, 'answers.json'), indent=4, orient='records')

In [29]:
df_question_samples = pd.read_json(
    os.path.join(path_so_sampled, 'questions.json'))
df_answer_samples = pd.read_json(os.path.join(path_so_sampled, 'answers.json'))
df_question_samples.loc[lambda x: ~x['Question_exclusive_tag'].isin(ignore_tools)].to_json(
    os.path.join(path_so_sampled, 'lean_questions.json'), indent=4, orient='records')
df_answer_samples.loc[lambda x: ~x['Question_exclusive_tag'].isin(ignore_tools)].to_json(
    os.path.join(path_so_sampled, 'lean_answers.json'), indent=4, orient='records')